In [9]:
import datetime


def delta_t(year, month, apply_correction=False):
    y = year + (month - 0.5) / 12

    if year < -500:
        u = (y - 1820) / 100
        delta_t = -20 + 32 * u**2
    elif year < 500:
        u = y / 100
        delta_t = (
            10583.6
            - 1014.41 * u
            + 33.78311 * u**2
            - 5.952053 * u**3
            - 0.1798452 * u**4
            + 0.022174192 * u**5
            + 0.0090316521 * u**6
        )
    elif year < 1600:
        u = (y - 1000) / 100
        delta_t = (
            1574.2
            - 556.01 * u
            + 71.23472 * u**2
            + 0.319781 * u**3
            - 0.8503463 * u**4
            - 0.005050998 * u**5
            + 0.0083572073 * u**6
        )
    elif year < 1700:
        t = y - 1600
        delta_t = 120 - 0.9808 * t - 0.01532 * t**2 + t**3 / 7129
    elif year < 1800:
        t = y - 1700
        delta_t = (
            8.83
            + 0.1603 * t
            - 0.0059285 * t**2
            + 0.00013336 * t**3
            - t**4 / 1174000
        )
    elif year < 1860:
        t = y - 1800
        delta_t = (
            13.72
            - 0.332447 * t
            + 0.0068612 * t**2
            + 0.0041116 * t**3
            - 0.00037436 * t**4
            + 0.0000121272 * t**5
            - 0.0000001699 * t**6
            + 0.000000000875 * t**7
        )
    elif year < 1900:
        t = y - 1860
        delta_t = (
            7.62
            + 0.5737 * t
            - 0.251754 * t**2
            + 0.01680668 * t**3
            - 0.0004473624 * t**4
            + t**5 / 233174
        )
    elif year < 1920:
        t = y - 1900
        delta_t = (
            -2.79
            + 1.494119 * t
            - 0.0598939 * t**2
            + 0.0061966 * t**3
            - 0.000197 * t**4
        )
    elif year < 1941:
        t = y - 1920
        delta_t = 21.20 + 0.84493 * t - 0.076100 * t**2 + 0.0020936 * t**3
    elif year < 1961:
        t = y - 1950
        delta_t = 29.07 + 0.407 * t - t**2 / 233 + t**3 / 2547
    elif year < 1986:
        t = y - 1975
        delta_t = 45.45 + 1.067 * t - t**2 / 260 - t**3 / 718
    elif year < 2005:
        t = y - 2000
        delta_t = (
            63.86
            + 0.3345 * t
            - 0.060374 * t**2
            + 0.0017275 * t**3
            + 0.000651814 * t**4
            + 0.00002373599 * t**5
        )
    elif year < 2050:
        t = y - 2000
        delta_t = 62.92 + 0.32217 * t + 0.005589 * t**2
    elif year < 2150:
        delta_t = -20 + 32 * ((y - 1820) / 100) ** 2 - 0.5628 * (2150 - y)
    else:
        u = (y - 1820) / 100
        delta_t = -20 + 32 * u**2

    if apply_correction:
        delta_t -= -0.000012932 * (y - 1955) ** 2

    return delta_t


def ymd(jd:float):
    k = jd + 68569
    n = 4 * k / 146097
    k = k - (146097 * n + 3) / 4
    m = 4000 * (k + 1) / 1461001
    k = k - 1461 * m / 4 + 31
    month = (80 * k / 2447)
    day = (k - 2447 * month / 80)
    k = month / 11
    month = month + 2 - 12 * k
    year = 100 * (n - 49) + m + k
    return int(year), int(month), int(day)
    

def leap_year(year):
    """Leap year or not in the Gregorian calendar."""
    x = year % 4
    y = year % 100
    z = year % 400

    # Divisible by 4 and,
    # either not divisible by 100 or divisible by 400.
    return not x and (y or not z)
def jd2fyr(jd):
    day_table = (
        [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365],
        [0, 31, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335, 366]
    )
    
    year, month, day = ymd(jd + 0.5)
  
    
    table = day_table[leap_year(year)]
    
    
    
    
    return year + ((table[month - 1] + day - 1) ) / table[-1]



def delta_t2(jd):
    a = [
        62.96117620920749,
        40.72414272333056,
        -86.78906680769823,
        245.6252926768666,
        295.8460515505873,
        -851.1075983781398
    ]
    fy = jd2fyr(jd)
    x = (fy - 2000.0) / 100.0
    y = a[5]
    i = 4
    while i>=0:
        y = y * x + a[i]
        i -= 1
    return y



def delta_t3(year, month, day):
    a =    [
        62.96117620920749,
        40.72414272333056,
        -86.78906680769823,
        245.6252926768666,
        295.8460515505873,
        -851.1075983781398
    ]
    day_table = (
        [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365],
        [0, 31, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335, 366]
    )
    table = day_table[leap_year(year)]
    fy = year + ((table[month - 1] + day - 1) ) / table[-1]
    x = (fy - 2000.0) / 100.0
    y = a[5]
    i = 4
    while i>=0:
        y = y * x + a[i]
        i -= 1
    return y


now = datetime.datetime(1950, 1, 1)


year = now.year
month = now.month
day = now.day


jd12h = (
    day - 32075 + 1461 * (year + 4800 + (month - 14) / 12) / 4 + 367 * (month - 2 - (month - 14) / 12 * 12) / 12 - 3 * ((year + 4900 + (month - 14) / 12) / 100) / 4
)
print(now.timestamp() / 86400 + 2440588.5 - jd12h)

def delta_t4(year):
    t = (year-1820)/100
    return -20 + 32 * t**2

def julian_day(year, month, day):
    num = (month - 14) / 12
    jd = day - 32075 + 1461 * (year + 4800 + num) / 4
    jd += 367 * (month - 2 - num * 12) / 12 - 3 * ((year + 4900 + num) / 100) / 4
    return jd

from nzthermo import _c

years = list(range(1900, 2020))
# %timeit delta_t2(julian_day(2014, 1, 1))
# %timeit delta_t3(2014, 1, 1)
# delta_t4(2014)

# %timeit _c.delta_t(years, 1)

[delta_t(y, 1) for y in years]

x = _c.delta_t(1970, 1)
x

0.30437499983236194


40.23467888274606

array([68.54577795])

In [ ]:

year = now.year
month = now.month
day = now.day
seconds = now.second
minute = now.minute
hour = now.hour
fday = (((seconds / 60.0) + minute) / 60.0) + hour / 24.0
jd12h = (
    day - 32075 + 1461 * (year + 4800 + (month - 14) / 12) / 4 + 367 * (month - 2 - (month - 14) / 12 * 12) / 12 - 3 * ((year + 4900 + (month - 14) / 12) / 100) / 4
)
jd = fday-0.5 + jd12h
jd 

delta_t2(2459696.8819444445)

In [ ]:
import math



MJD_0 = 2400000.5
MJD_JD2000 = 51544.5




    

def julian_day(year, month, day):
    num = (month - 14) / 12
    jd = day - 32075 + 1461 * (year + 4800 + num) / 4
    jd += 367 * (month - 2 - num * 12) / 12 - 3 * ((year + 4900 + num) / 100) / 4
    return jd

now = datetime.datetime(1950, 1, 1)
year = now.year
month = now.month
day = now.day
jd = julian_day(now.year, now.month, now.day)
num = (month - 14) / 12
jd2 = day - 32075 + 1461 * (year + 4800 + num) / 4
jd2 += 367 * (month - 2 - num * 12) / 12 - 3 * ((year + 4900 + num) / 100) / 4
jd2,jd, day - 32075 + 1461 * (year + 4800 + num) / 4 + 367 * (month - 2 - num * 12) / 12 - 3 * ((year + 4900 + num) / 100) / 4